In [1]:
import torch
import random
import Args
import MMDLoss
import Model
import Data
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import time
import copy
import KMeansPP

In [2]:
l1 = [1, 2 , 3]
l2 = [4, 5, 6]


In [8]:
model = Model.init_model('cifar10')
model.state_dict()['fc4.bias']

tensor([ 0.2055,  0.0710,  0.1412,  0.1623,  0.0821, -0.1355,  0.2009,  0.0707,
         0.2173,  0.1765])

In [2]:
args = Args.Arguments()

In [3]:
dataset = datasets.CIFAR10(root='data', train=True, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
dataloader = DataLoader(dataset, batch_size=args.batch_size, shuffle=False)

In [4]:
model1 = Model.Cifar10Model()
model2 = Model.Cifar10Model()
optimizer1 = optim.SGD(model1.parameters(), lr=0.02)
optimizer2 = optim.SGD(model2.parameters(), lr=0.02)
print(KMeansPP.get_cos_dis_single_layer(model1.fc4.state_dict()['weight'], model2.fc4.state_dict()['weight']))

tensor([ 0.2828, -0.2038,  0.1429, -0.1397, -0.1297,  0.2316,  0.0203, -0.2688,
        -0.4168,  0.1342])


In [5]:
model1.train(True)
model2.train(True)
device = torch.device('cuda:0')
model1.to(device)
model2.to(device)
for epoch in range(10):
    dict1 = copy.deepcopy(model1.fc4.state_dict()["weight"])
    dict2 = copy.deepcopy(model2.fc4.state_dict()["weight"])
    for data, label in dataloader:
        data = data.to(device)
        label = label.to(device)
        optimizer1.zero_grad()
        pred = model1(data)

        loss1 = F.nll_loss(pred, label)

        # loss1 = F.nll_loss(pred, label)

        loss1.backward()

        optimizer1.step()

        ##
        optimizer2.zero_grad()
        pred = model2(data)

        loss2 = F.nll_loss(pred, label)

        # loss1 = F.nll_loss(pred, label)

        loss2.backward()

        optimizer2.step()
    print(KMeansPP.get_cos_dis_single_layer(model1.fc4.state_dict()['weight'].sub(dict1), model2.fc4.state_dict()['weight'].sub(dict2)))

tensor([ 0.1970, -0.3423,  0.1264, -0.2713, -0.2158,  0.1484, -0.2164, -0.3390,
        -0.2090, -0.2919], device='cuda:0')
tensor([-0.3287, -0.0933, -0.5173, -0.0747, -0.4996,  0.0162, -0.3238, -0.2581,
        -0.4338, -0.0927], device='cuda:0')
tensor([-0.2421, -0.3746, -0.1047, -0.4276,  0.0694, -0.1826, -0.2595, -0.3783,
        -0.0200,  0.0808], device='cuda:0')
tensor([-0.0542, -0.3334, -0.0824, -0.4151, -0.0150, -0.2531, -0.3375, -0.2910,
        -0.1728,  0.5150], device='cuda:0')
tensor([ 0.0166, -0.1043, -0.2898, -0.3010, -0.0662, -0.0517, -0.3832, -0.4422,
        -0.2733,  0.4385], device='cuda:0')
tensor([ 0.1032, -0.1636, -0.2590,  0.0090, -0.0240,  0.2124, -0.2982, -0.3355,
        -0.2318,  0.3556], device='cuda:0')
tensor([ 0.0275, -0.2269, -0.2670,  0.0576,  0.0233, -0.2501, -0.3826, -0.3869,
        -0.1182,  0.3069], device='cuda:0')
tensor([ 0.1198, -0.0388, -0.3286, -0.0578,  0.0376, -0.1585, -0.3504, -0.3504,
         0.0390,  0.1918], device='cuda:0')


KeyboardInterrupt: 